In [39]:
import cnvpytor
import subprocess

print(cnvpytor.__version__)

1.3.1


In [40]:
filename = "test_with_del__1_formated.fa"
command = f"cat {filename} | wc -l"
result = subprocess.check_output(command, shell=True)
line_count = int(result.strip())
line_count

114971

In [41]:
read_count = line_count * 50 * 30 / 202
read_count = round(read_count)
read_count

853745

In [42]:
command = f"wgsim -d 300 -N {read_count} -1 101 -2 101 {filename} frag1Copy_test.fastq frag2Copy_test.fastq"
result = subprocess.check_output(command, shell=True)
print(result)

[wgsim] seed = 1706525133
[wgsim_core] calculating the total length of the reference sequence...
[wgsim_core] 1 sequences, total length: 5748464


b'chr22\t150\tC\tT\t-\nchr22\t777\tT\tG\t-\nchr22\t1431\tGTC\t-\t-\nchr22\t1715\tT\tY\t+\nchr22\t5026\tG\tR\t+\nchr22\t7966\tA\tT\t-\nchr22\t8321\tA\tT\t-\nchr22\t10519\tG\tR\t+\nchr22\t11195\tC\tM\t+\nchr22\t15633\t-\tAG\t-\nchr22\t16698\tT\tA\t-\nchr22\t17459\tC\tM\t+\nchr22\t18357\tC\tT\t-\nchr22\t19462\tC\tY\t+\nchr22\t19950\tT\tK\t+\nchr22\t20263\tT\tK\t+\nchr22\t21236\tT\tW\t+\nchr22\t21607\tA\tM\t+\nchr22\t22653\tA\tR\t+\nchr22\t22896\tG\tK\t+\nchr22\t24212\tT\tW\t+\nchr22\t25264\tT\tC\t-\nchr22\t27989\tG\tA\t-\nchr22\t30082\tC\tS\t+\nchr22\t30157\tA\tW\t+\nchr22\t31597\tT\tY\t+\nchr22\t31629\tA\tG\t-\nchr22\t32192\tT\tG\t-\nchr22\t32448\tT\tK\t+\nchr22\t32515\tC\tT\t-\nchr22\t33103\tA\tM\t+\nchr22\t35204\tG\tR\t+\nchr22\t35622\tG\tC\t-\nchr22\t38728\tT\tW\t+\nchr22\t39217\tA\t-\t-\nchr22\t40331\tC\tT\t-\nchr22\t43945\tT\tG\t-\nchr22\t44865\tG\tR\t+\nchr22\t45771\tA\tM\t+\nchr22\t46407\tT\tK\t+\nchr22\t47714\tC\tM\t+\nchr22\t48700\t-\tT\t+\nchr22\t49908\tA\tG\t-\nchr22\t53187\tA

In [43]:
# bowtie related stuff
!bowtie2-build reference.fa bias
!bowtie2 -p 4 -k 15 --no-mixed -x bias -1 frag1Copy_test.fastq -2 frag2Copy_test.fastq -S biasOut.sam

Settings:
  Output files: "bias.*.bt2"
  Line rate: 6 (line is 64 bytes)
  Lines per side: 1 (side is 64 bytes)
  Offset rate: 4 (one in 16)
  FTable chars: 10
  Strings: unpacked
  Max bucket size: default
  Max bucket size, sqrt multiplier: default
  Max bucket size, len divisor: 4
  Difference-cover sample period: 1024
  Endianness: little
  Actual local endianness: little
  Sanity checking: disabled
  Assertions: disabled
  Random seed: 0
  Sizeofs: void*:8, int:4, long:8, size_t:8
Input files DNA, FASTA:
  reference.fa
Building a SMALL index
Reading reference sizes
  Time reading reference sizes: 00:00:00
Calculating joined length
Writing header
Reserving space for joined string
Joining reference sequences
  Time to join reference sequences: 00:00:01
bmax according to bmaxDivN setting: 12826141
Using parameters --bmax 9619606 --dcv 1024
  Doing ahead-of-time memory usage test
  Passed!  Constructing with these parameters: --bmax 9619606 --dcv 1024
Constructing suffix-array element

In [44]:
# sam to bam file conversion
!samtools sort biasOut.sam -o biasOut_sorted.bam
!samtools index biasOut_sorted.bam

[bam_sort_core] merging from 1 files and 1 in-memory blocks...


In [45]:
app = cnvpytor.Root("test.pytor", create=True, max_cores=8)

In [46]:
app.rd(["biasOut_sorted.bam"])

In [47]:
app.calculate_histograms([100, 1000, 10000, 100000, 1000000, 10000000])

In [48]:
app.partition([100, 1000, 10000, 100000, 1000000, 10000000])

In [49]:
calls = app.call([100, 1000, 10000, 100000, 1000000, 10000000], print_calls=True)

duplication	chr22:643201-644800	1600	1.4963	5.899373e-06	2.350791e-57	1.000000e+00	1.000000e+00	0.0000	1.0000	0
duplication	chr22:1306101-1325100	19000	1.4074	0.000000e+00	1.695385e-271	0.000000e+00	5.333755e-242	0.0000	1.0000	0
duplication	chr22:2926201-2964700	38500	3.0634	0.000000e+00	4.776568e-77	0.000000e+00	1.365097e-72	0.0000	1.0000	0
duplication	chr22:4546401-4713300	166900	1.6779	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6655001-6661800	6800	2.4462	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:6848301-6854600	6300	1.6836	0.000000e+00	9.416501e-242	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:8038501-8047700	9200	1.8771	0.000000e+00	7.395594e-09	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:9424201-9461600	37400	1.4982	0.000000e+00	0.000000e+00	0.000000e+00	0.000000e+00	0.0000	1.0000	0
duplication	chr22:13230601-13395900	165300	1.4175	0.000000e+00	0.000000e+00	0.0000

In [50]:
for bin_size in calls:
    print("Bin size: ", bin_size)

Bin size:  100
Bin size:  1000
Bin size:  10000
Bin size:  100000
Bin size:  1000000
Bin size:  10000000


In [51]:
print(f"{filename=}, Histogram Size: 100")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 100
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,643201,644800,1600,1.4963459516971191,0.0
duplication,chr22,1306101,1325100,19000,1.407350210271597,0.0
duplication,chr22,2926201,2964700,38500,3.0633784118329133,0.0
duplication,chr22,4546401,4713300,166900,1.6778826070145598,0.0
duplication,chr22,6655001,6661800,6800,2.4462204255276,0.0
duplication,chr22,6848301,6854600,6300,1.683623682408784,0.0
duplication,chr22,8038501,8047700,9200,1.8770808436771054,0.0
duplication,chr22,9424201,9461600,37400,1.4982458710233642,0.0
duplication,chr22,13230601,13395900,165300,1.4174972184355727,0.0
duplication,chr22,13449001,13453700,4700,1.3335185279920345,0.0
duplication,chr22,13480901,13512900,32000,4.261177598198237,0.0
duplication,chr22,15242601,15246400,3800,3.0395276623233167,0.0
duplication,chr22,18662501,18664600,2100,1.4512122496482287,0.0
duplication,chr22,18664801,18665900,1100,1.4940410586228614,0.0
duplicatio

In [52]:
print(f"{filename=}, Histogram Size: 1000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 1000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,1306001,1325000,19000,1.4042088530014698,0.0
duplication,chr22,2926001,2963000,37000,3.1388230815004876,0.0
duplication,chr22,4546001,4713000,167000,1.67407368005837,0.0
duplication,chr22,6655001,6662000,7000,2.40618304134782,0.0
duplication,chr22,8040001,8048000,8000,1.9583731556139472,0.0
duplication,chr22,9424001,9462000,38000,1.49021530224858,0.0
duplication,chr22,13231001,13396000,165000,1.4147504237700577,0.0
duplication,chr22,13481001,13513000,32000,4.24823598895345,0.0
duplication,chr22,18705001,18881000,176000,2.721427783960811,0.00014096247401351243
duplication,chr22,19259001,19268000,9000,3.238220444443018,0.0
duplication,chr22,21248001,21381000,133000,1.4621731699118239,0.0001327518800985019
duplication,chr22,21533001,21548000,15000,1.2868803426862336,0.0026642496154691275
duplication,chr22,21582001,21603000,21000,1.632007759637733,0.0001719878670

In [53]:
print(f"{filename=}, Histogram Size: 10,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 10,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,4550001,4710000,160000,1.670786407678505,0.0
duplication,chr22,6610001,7000000,390000,1.09903727113158,0.0
duplication,chr22,7960001,8040000,80000,1.0710401627029806,0.0
duplication,chr22,13230001,13410000,180000,1.3828553788216882,0.0
duplication,chr22,18710001,18880000,170000,2.711740774394776,0.00014548353802217994
duplication,chr22,21250001,21380000,130000,1.4441624989121837,0.00013293723463421914
duplication,chr22,25150001,25310000,160000,1.2639061965097642,0.0001291900307054689
duplication,chr22,28470001,28600000,130000,3.0516745588128393,4.274676951631341e-05
duplication,chr22,30800001,30980000,180000,2.822111787051224,2.5984733968793333e-05
duplication,chr22,31150001,31360000,210000,1.402873973359699,4.966071092854888e-05
duplication,chr22,31860001,32250000,390000,1.2548729280317517,9.4367763104374e-05
duplication,chr22,35940001,36200000,260000,2.16

In [54]:
print(f"{filename=}, Histogram Size: 100,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[100000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 100,000
type,chromosome_name,start,end,size,cnv_level,q0
duplication,chr22,3000001,6700000,3700000,1.0807974510014813,0.0
duplication,chr22,45700001,51400000,5700000,1.572989350815511,7.324800844307934e-05


In [55]:
print(f"{filename=}, Histogram Size: 1,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[1000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 1,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [56]:
print(f"{filename=}, Histogram Size: 10,000,000")
print("type,chromosome_name,start,end,size,cnv_level,q0")

for call in calls[10000000]:
    print(call[0], call[1], call[2], call[3], call[4], call[5], call[10], sep=",")

filename='test_with_del__1_formated.fa', Histogram Size: 10,000,000
type,chromosome_name,start,end,size,cnv_level,q0


In [57]:
del app